- Data su zo stranky www.soga.sk, jedna sa o vsetky aukcie od roku 2004 az po 2024, cca 100 aukcii a 21,000 drazenych diel

In [66]:
import sqlite3
import pandas as pd
import pandasql as ps
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import output_notebook

In [67]:
# Connect to the SQLite database
conn = sqlite3.connect('soga.db')

# Query to select data from the database
query = """
SELECT 
    UPPER(SUBSTR(autor, 1, INSTR(autor, '(') - 1)) AS autor,
    a.nazov, 
    vydrazene,
    CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) AS konecna_cena,
    CAST(REPLACE(vyvolavacia_cena, ' ', '') AS INTEGER) AS vyvolavacia_cena,
    rok,
    technika,
    datum,
    url_soga
FROM diela a 
LEFT JOIN aukcie b ON a.id_aukcie = b.id 
WHERE TRUE 
-- AND autor like '%FILKO%'
ORDER BY CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) DESC;
"""

# Load data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga
0,DOMENI...,Chlape...,1,531103.0,663878.0,okolo ...,olej n...,04. Ok...,https:...
1,BENKA ...,Po žatve,1,180000.0,NaN,1922-24,Olej n...,13. No...,https:...
2,BENKA ...,Nad hĺ...,1,180000.0,NaN,1948,Olej n...,22. Má...,https:...
3,BENKA ...,Sloven...,1,176000.0,NaN,1938 - 40,olej n...,15. No...,https:...
4,WARHOL...,Margar...,1,170000.0,NaN,1985,Farebn...,13. No...,https:...


# Priprava dat

In [68]:
from unidecode import unidecode  # Import the unidecode function
import re


# Custom function to standardize and reorder names
def standardize_and_order_name(name):
    # Convert accented characters to ASCII equivalents
    name = unidecode(name)
    name= name.upper()

    # Split the name into parts
    name_parts = name.split()

    # Reorder the name parts alphabetically
    standardized_name = ' '.join(sorted(name_parts))
    
    

    return standardized_name


def extract_first_year(date_string):
    # Regular expression pattern to match a year
    year_pattern = re.compile(r'\b\d{4}\b')

    # Find the first match in the string
    match = year_pattern.search(date_string)

    # Extract and return the matched year
    if match:
        return int(match.group())

    return None  # Return None if no year is found


In [69]:
df['autor']=df['autor'].apply(standardize_and_order_name)

In [70]:
df['olej']=df['technika'].apply(lambda x: 'olej' in x.lower())

In [71]:
df['rok_vytvorenia']=df['rok'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)
df['rok_drazby']=df['datum'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)

In [72]:
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga,olej,rok_vytvorenia,rok_drazby
0,DOMENI...,Chlape...,1,531103.0,663878.0,okolo ...,olej n...,04. Ok...,https:...,True,1615,2005
1,BENKA ...,Po žatve,1,180000.0,NaN,1922-24,Olej n...,13. No...,https:...,True,1922,2012
2,BENKA ...,Nad hĺ...,1,180000.0,NaN,1948,Olej n...,22. Má...,https:...,True,1948,2018
3,BENKA ...,Sloven...,1,176000.0,NaN,1938 - 40,olej n...,15. No...,https:...,True,1938,2011
4,ANDY W...,Margar...,1,170000.0,NaN,1985,Farebn...,13. No...,https:...,False,1985,2012


# Analyza

## Top 50

### Top 50 najdrazsich autorov podla priemernej ceny

In [73]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
--and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,ANDY W...,11,47953.0,465.0,170000.0
1,SKUTETZKY,10,46090.0,700.0,120000.0
2,FOLTYN...,14,35791.0,320.0,162000.0
3,CYPRIA...,14,32980.0,200.0,170000.0
4,JAKOBY...,40,28312.0,266.0,120000.0
5,EMIL F...,6,24655.0,199.0,140000.0
6,BENKA ...,183,20559.0,200.0,180000.0
7,JAKABC...,6,15297.0,1228.0,38500.0
8,HALA JAN,165,13705.0,66.0,112000.0
9,VIERA ...,14,11259.0,3600.0,18600.0


### Top 50 najdrazsich autorov - oleje

In [74]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,FULLA ...,14,66586.0,14539.0,160000.0
1,SKUTETZKY,8,57438.0,9500.0,120000.0
2,FOLTYN...,11,45411.0,2357.0,162000.0
3,GALAND...,10,39854.0,5145.0,120000.0
4,BENKA ...,116,31176.0,1162.0,180000.0
5,JAKOBY...,36,30361.0,3153.0,120000.0
6,LADISL...,142,21060.0,1992.0,130000.0
7,HALA JAN,102,21021.0,500.0,112000.0
8,ENDRE ...,9,16233.0,3500.0,34000.0
9,ALEXAN...,79,15976.0,800.0,90000.0


### Top 50 najdrazsich diel

In [75]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,DOMENICHINO,Chlapec s dalmatínom,531103.0,1615,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/domenichino-domenico-zampieri-chlapec-s-dalmatinom-9260
1,BENKA MARTIN,Po žatve,180000.0,1922,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/benka-martin-po-zatve-32046
2,BENKA MARTIN,Nad hĺbkou doliny,180000.0,1948,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/140-vecerna-aukcia/benka-martin-nad-hlbkou-doliny-67594
3,BENKA MARTIN,Slovenská rodina,176000.0,1938,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/benka-martin-slovenska-rodina-29395
4,ANDY WARHOL,"Margaréta II., dánska kráľovná, z cyklu Vládnuce kráľovné (portfólio 4 sieťotlačí)",170000.0,1985,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/warhol-andy-margareta-ii-danska-kralovna-z-cyklu-vladnuce-kralovne-portfolio-4-sietotlaci-32127
5,CYPRIAN MAJERNIK,Utečenci,170000.0,1939,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/majernik-cyprian-utecenci-29404
6,FOLTYN FRANTISEK,Krajina od Mukačeva,162000.0,1922,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/foltyn-frantisek-krajina-od-mukaceva-29412
7,FULLA LUDOVIT,Blumentál v zime,160000.0,1944,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/fulla-ludovit-blumental-v-zime-29378
8,BENKA MARTIN,Z Detvy,146000.0,1932,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/benka-martin-z-detvy-62558
9,EMIL FILLA,Zátišie s azalkou,140000.0,1947,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/filla-emil-zatisie-s-azalkou-29380


### Top 50 najdrazsich diel - nie oleje

In [76]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
and not olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,ANDY WARHOL,"Margaréta II., dánska kráľovná, z cyklu Vládnuce kráľovné (portfólio 4 sieťotlačí)",170000.0,1985,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/warhol-andy-margareta-ii-danska-kralovna-z-cyklu-vladnuce-kralovne-portfolio-4-sietotlaci-32127
1,ANDY WARHOL,Hans Christian Andersen (portfólio 4 sieťotlačí),120000.0,1987,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/warhol-andy-hans-christian-andersen-portfolio-4-sietotlaci-29353
2,ANDY WARHOL,Kladivo a kosák (portfólio 4 sieťotlačí),104000.0,1977,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/102-jarna-aukcia-vytvarnych-diel/warhol-andy-kladivo-a-kosak-portfolio-4-sietotlaci-29984
3,,Prsteň s briliantom,80000.0,None,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/138-zimna-aukcia-vytvarnych-diel-a-starozitnosti/prsten-s-briliantom-66963
4,ENDRE NEMES,Kompozícia,57000.0,1950,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/nemes-endre-kompozicia-29376
5,CORNELIS DE WAEL,BOJOVÁ SCÉNA,53110.0,None,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/73-aukcia-umeleckeho-remesla-a-starozitnosti/cornelis-de-wael-bojova-scena-14029
6,ALBIN BRUNOVSKY,Žena s klobúkom,53000.0,1981,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/148-aukcia-vytvarnych-diel/brunovsky-albin-zena-s-klobukom-69213
7,,ROMÁNSKA TRÓNIACA MADONA S DIEŤAŤOM,51451.0,None,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/73-aukcia-umeleckeho-remesla-a-starozitnosti/romanska-troniaca-madona-s-dietatom-14022
8,ANDREJ BARCIK,Veľká sediaca postava,50000.0,1982,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/128-vecerna-aukcia/barcik-andrej-velka-sediaca-postava-63302
9,BARTFAY TIBOR,Pegas,48000.0,1990,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/86-letna-aukcia-vytvarnych-diel-a-starozitnosti/bartfay-tibor-pegas-19373


### Top 50 najdrazsich diel - Pasteka

In [77]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%PASTEKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,MILAN PASTEKA,Vták,50000.0,1973,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/153-letna-aukcia-diel-vytvarneho-umenia/pasteka-milan-vtak-75322
1,MILAN PASTEKA,Čakáreň,44000.0,1980,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/pasteka-milan-cakaren-29371
2,MILAN PASTEKA,Sediaca (Na červenej stoličke),42000.0,1973,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/108-zimna-aukcia-vytvarnych-diel-a-starozitnosti/pasteka-milan-sediaca-na-cervenej-stolicke-41020
3,MILAN PASTEKA,V izbe,38000.0,1975,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/91-aukcia-sucasneho-umenia/pasteka-milan-v-izbe-21937
4,MILAN PASTEKA,Torzo,30000.0,1990,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/116-vecerna-aukcia/pasteka-milan-torzo-50391
5,MILAN PASTEKA,Zátišie,24000.0,1974,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/pasteka-milan-zatisie-32098
6,MILAN PASTEKA,Biely kôň,21000.0,1982,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/143-vecerna-aukcia/pasteka-milan-biely-kon-68229
7,MILAN PASTEKA,Sediaci,21000.0,1987,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/pasteka-milan-sediaci-75481
8,MILAN PASTEKA,Skupina,18000.0,1961,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/148-aukcia-vytvarnych-diel/pasteka-milan-skupina-69253
9,MILAN PASTEKA,Spojenie,17000.0,1989,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/milan-pasteka-spojenie-75775


### Top 50 najdrazsich diel - Binder

In [78]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BINDER %'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BINDER ERIK,O autách a vtákoch,6700.0,2011,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/binder-erik-o-autach-a-vtakoch-29351
1,BINDER ERIK,Možnosti ostrova,6200.0,2008,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/147-vecerna-aukcia/binder-erik-moznosti-ostrova-69088
2,BINDER ERIK,"Ja somár, načo som sa snažil?!",5000.0,2014,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/binder-erik-ja-somar-naco-som-sa-snazil-52982
3,BINDER ERIK,Ako by nás jedna mater mala,4900.0,2012,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/binder-erik-ako-by-nas-jedna-mater-mala-62619
4,BINDER ERIK,Last Minute,4800.0,2015,2020,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/150-on-line-aukcia-vytvarnych-diel/erik-binder-last-minute-69553
5,BINDER ERIK,Galaktická cesta,4000.0,2010,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/erik-binder-galakticka-cesta-75764
6,BINDER ERIK,"O Elektrike (A tak si volám taxi, okolo lietajú smsky a faxy)",3700.0,2013,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/binder-erik-o-elektrike-a-tak-si-volam-taxi-okolo-lietaju-smsky-a-faxy-51356
7,BINDER ERIK,Športovo-fajčiarsko-biblický motív,2800.0,2008,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/116-vecerna-aukcia/binder-erik-sportovo-fajciarsko-biblicky-motiv-50432
8,BINDER ERIK,Športovo-fajčiarsko-biblický motív,2800.0,2008,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/143-vecerna-aukcia/binder-erik-sportovo-fajciarsko-biblicky-motiv-68267
9,BINDER ERIK,Dojazd,2556.0,2008,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/79-aukcia-sucasneho-slovenskeho-umenia/erik-binder-dojazd-17050


### Top 50 najdrazsich diel - Filko

In [79]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILKO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FILKO STANO,Oltár súčasnosti,45000.0,1964,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/147-vecerna-aukcia/filko-stano-oltar-sucasnosti-69141
1,FILKO STANO,Eva,24000.0,1957,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/119-vecerna-aukcia/filko-stano-eva-51758
2,FILKO STANO,"Asociácie (Priestor, energia, odvaha, plodnosť)",18000.0,1970,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/128-vecerna-aukcia/filko-stano-asociacie-priestor-energia-odvaha-plodnost-63305
3,FILKO STANO,Zbojníci,15700.0,1962,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/110-vecerna-aukcia/filko-stano-zbojnici-43457
4,FILKO STANO,Oltár súčasnosti,15000.0,1964,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/86-letna-aukcia-vytvarnych-diel-a-starozitnosti/filko-stano-oltar-sucasnosti-19367
5,FILKO STANO,Mesto,12800.0,1963,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/filko-stano-mesto-62616
6,FILKO STANO,Zbojníci,11500.0,1958,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/134-vecerna-aukcia/filko-stano-zbojnici-66014
7,FILKO STANO,Figurálna kompozícia,11200.0,1958,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/filko-stano-figuralna-kompozicia-51328
8,FILKO STANO,"5. dimenzia - čakra bielej metafyziky, ontologická transcendencia",10000.0,1959,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/137-vecerna-aukcia/filko-stano-5-dimenzia-cakra-bielej-metafyziky-ontologicka-transcendencia-66700
9,FILKO STANO,Čierne na bielom - biele na bielom,9200.0,1959,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/131-vecerna-aukcia/filko-stano-cierne-na-bielom-biele-na-bielom-65109


### Top 50 najdrazsich diel - Laluha

In [124]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%LALUHA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,LALUHA MILAN,Traja chlapi,64000.0,1967,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/116-vecerna-aukcia/laluha-milan-traja-chlapi-50401
1,LALUHA MILAN,Odchádzajúca žena,35000.0,1968,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/119-vecerna-aukcia/laluha-milan-odchadzajuca-zena-51766
2,LALUHA MILAN,Ženy z Mičinej,31000.0,1965,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/90-jarna-aukcia-vytvarnych-diel/laluha-milan-zeny-z-micinej-21491
3,LALUHA MILAN,Žena s batohom,28000.0,1965,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/laluha-milan-zena-s-batohom-32106
4,LALUHA MILAN,Žena,17900.0,1970,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/laluha-milan-zena-29373
5,LALUHA MILAN,Kytica,17600.0,1959,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/129-letna-aukcia-vytvarnych-diel-a-starozitnosti/laluha-milan-kytica-63514
6,LALUHA MILAN,Akt,15700.0,1970,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/laluha-milan-akt-52950
7,LALUHA MILAN,Päť chlapov,15600.0,1969,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/milan-laluha-pat-chlapov-75798
8,LALUHA MILAN,Sediaci akt,15100.0,1965,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/131-vecerna-aukcia/laluha-milan-sediaci-akt-65094
9,LALUHA MILAN,Motív z Moravian,15000.0,1965,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/131-vecerna-aukcia/laluha-milan-motiv-z-moravian-65088


### Top 50 najdrazsich diel - Benka

In [125]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BENKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BENKA MARTIN,Po žatve,180000.0,1922,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/benka-martin-po-zatve-32046
1,BENKA MARTIN,Nad hĺbkou doliny,180000.0,1948,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/140-vecerna-aukcia/benka-martin-nad-hlbkou-doliny-67594
2,BENKA MARTIN,Slovenská rodina,176000.0,1938,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/benka-martin-slovenska-rodina-29395
3,BENKA MARTIN,Z Detvy,146000.0,1932,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/benka-martin-z-detvy-62558
4,BENKA MARTIN,V jeseni - Turiec,120000.0,1947,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/110-vecerna-aukcia/benka-martin-v-jeseni-turiec-43403
5,BENKA MARTIN,Pri oberačke ovocia,120000.0,1938,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/156-zimna-aukcia/martin-benka-pri-oberacke-ovocia-75886
6,BENKA MARTIN,Drevári,107000.0,1940,2021,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/151-vianocna-aukcia/benka-martin-drevari-69659
7,BENKA MARTIN,Z bošáckeho kraja,98000.0,1943,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/113-vecerna-aukcia/benka-martin-z-bosackeho-kraja-47986
8,BENKA MARTIN,Na pole (Liptov),90000.0,1930,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/119-vecerna-aukcia/benka-martin-na-pole-liptov-51740
9,BENKA MARTIN,Z tisovského kraja,87000.0,1946,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/110-vecerna-aukcia/benka-martin-z-tisovskeho-kraja-43402


### Top 50 najdrazsich diel - Fulla

In [126]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FULLA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FULLA LUDOVIT,Blumentál v zime,160000.0,1944,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/fulla-ludovit-blumental-v-zime-29378
1,FULLA LUDOVIT,Ukrižovanie,106500.0,1939,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-ukrizovanie-24250
2,FULLA LUDOVIT,Matka s dieťaťom,106000.0,1944,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/110-vecerna-aukcia/fulla-ludovit-matka-s-dietatom-43426
3,FULLA LUDOVIT,V poli na jeseň,99582.0,1947,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/80-letna-aukcia-vytvarnych-diel-a-starozitnosti/ludovit-fulla-v-poli-na-jesen-17375
4,FULLA LUDOVIT,Otčenáš,95000.0,1975,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/fulla-ludovit-otcenas-52930
5,FULLA LUDOVIT,Májové ráno,80000.0,1950,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-majove-rano-24249
6,FULLA LUDOVIT,Pod Tatrami (Senníky pod Kriváňom),73000.0,1949,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/139-jarna-aukcia-vytvarnych-diel/fulla-ludovit-pod-tatrami-senniky-pod-krivanom-67325
7,FULLA LUDOVIT,Kytica cyklámenov,56430.0,1930,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/81-jesenna-aukcia-vytvarnych-diel/ludovit-fulla-kytica-cyklamenov-18170
8,FULLA LUDOVIT,Perníková madona s anjelmi,48000.0,1965,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/101-zimna-aukcia-vytvarnych-diel-a-starozitnosti/fulla-ludovit-pernikova-madona-s-anjelmi-29540
9,FULLA LUDOVIT,Kostol Všetkých svätých v Ružomberku,47000.0,1940,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/148-aukcia-vytvarnych-diel/fulla-ludovit-kostol-vsetkych-svatych-v-ruzomberku-69278


### Top 50 najdrazsich diel - Klimo

In [128]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%KLIMO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,ALOJZ KLIMO,Štvorce a kruhy I.,5200.0,1967,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/100-jubilejna-vecerna-aukcia/klimo-alojz-stvorce-a-kruhy-i-29362
1,ALOJZ KLIMO,Križovatka I.,5000.0,1966,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/klimo-alojz-krizovatka-i-32118
2,ALOJZ KLIMO,Križovatky,4500.0,1966,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/klimo-alojz-krizovatky-75482
3,ALOJZ KLIMO,Križovatky,3900.0,1967,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/120-zimna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-krizovatky-52355
4,ALOJZ KLIMO,Križovatka,3700.0,1966,2020,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/149-on-line-aukcia-vytvarnych-diel/klimo-alojz-krizovatka-69452
5,ALOJZ KLIMO,Križovatka,3400.0,1965,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/105-letna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-krizovatka-30588
6,ALOJZ KLIMO,Križovatka,2800.0,1966,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/112-jesenna-aukcia-vytvarnych-diel/klimo-alojz-krizovatka-47274
7,ALOJZ KLIMO,Kompozícia,2700.0,1969,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/106-jesenna-aukcia-vytvarnych-diel/klimo-alojz-kompozicia-30997
8,ALOJZ KLIMO,Križovatka,2500.0,1967,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/132-zimna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-krizovatka-65668
9,ALOJZ KLIMO,Križovatka,2500.0,1966,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/130-jesenna-aukcia-vytvarnych-diel/klimo-alojz-krizovatka-65002


### Top 50 najdrazsich diel - Roller

In [129]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%ROLLER%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,PETER ROLLER,Architektúra,2855.0,1990,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/71-aukcia-sucasneho-slovenskeho-umenia/peter-roller-architektura-13717
1,PETER ROLLER,Objekt,1200.0,1980,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/138-zimna-aukcia-vytvarnych-diel-a-starozitnosti/roller-peter-objekt-66928
2,PETER ROLLER,Kameň,1162.0,None,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/56-aukcia-sucasneho-slovenskeho-umenia/peter-roller-kamen-8119
3,PETER ROLLER,Kalich.,450.0,2004,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/103-aukcia-sucasneho-umenia/roller-peter-kalich-30091
4,PETER ROLLER,Kompozícia,400.0,2012,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/145-aukcia-vytvarnych-diel/roller-peter-kompozicia-68850
5,PETER ROLLER,Svetlo monolitu.,350.0,2004,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/103-aukcia-sucasneho-umenia/roller-peter-svetlo-monolitu-30090
6,PETER ROLLER,Kompozícia,100.0,2011,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/127-jarna-aukcia-vytvarnych-diel/roller-peter-kompozicia-63200


### Top 50 najdrazsich diel - Fila

In [130]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FILA RUDOLF,Pokus o úlet,9000.0,1993,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/119-vecerna-aukcia/fila-rudolf-pokus-o-ulet-51774
1,FILA RUDOLF,Barvopád,7000.0,2002,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/121-jarna-aukcia-vytvarnych-diel/fila-rudolf-barvopad-52675
2,FILA RUDOLF,Za šrafami,7000.0,1967,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/fila-rudolf-za-srafami-75507
3,FILA RUDOLF,Ohyb,6800.0,1992,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/123-letna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-ohyb-61743
4,FILA RUDOLF,Replika (s Ingresom),6600.0,1999,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/111-letna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-replika-s-ingresom-45802
5,FILA RUDOLF,Akt-premaľba,6200.0,1975,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/102-jarna-aukcia-vytvarnych-diel/fila-rudolf-akt-premalba-30034
6,FILA RUDOLF,Vrstovník II.,5700.0,1968,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/108-zimna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-vrstovnik-ii-40961
7,FILA RUDOLF,Príbeh,5200.0,1967,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/126-zimna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-pribeh-62845
8,FILA RUDOLF,Laokonka,4500.0,1991,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/fila-rudolf-laokonka-32112
9,FILA RUDOLF,Kompozícia s vajíčkom,4400.0,1967,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fila-rudolf-kompozicia-s-vajickom-24000


## Najvyssi narast a pokles cien

### Najvyssi narast

In [134]:
from scipy.stats import linregress
pd.set_option('display.max_colwidth', None)

# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].agg(['count', 'mean']).reset_index()
df_avg = df_avg[df_avg['count']>=3][['autor', 'rok_drazby', 'mean']]

# Filter authors with at least 50 rows
author_counts = df_avg['autor'].value_counts()
selected_authors = author_counts[author_counts >= 5].index
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Calculate the slope for each author
slopes = []

for author in selected_authors:
    author_data = df_avg_filtered[df_avg_filtered['autor'] == author]
    slope, _, _, _, _ = linregress(author_data['rok_drazby'], author_data['mean'])
    slopes.append({'autor': author, 'priemerny_rocny_narast_v_cene': slope})

# Create a DataFrame for the slopes
slopes_df = pd.DataFrame(slopes)

# Display the slopes
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=False).reset_index(drop=True)[0:20]

,autor,priemerny_rocny_narast_v_cene
0,JAKOBY JULIUS,2635.941288
1,GALANDA MIKULAS,1221.123657
2,BENKA MARTIN,1205.305676
3,ALEXANDER BAZOVSKY MILOS,957.500711
4,HALA JAN,917.852904
5,GUDERNA LADISLAV,820.537313
6,GUSTAV MALLY,742.599351
7,FILKO STANO,501.621413
8,DOBES MILAN,500.550787
9,ALBIN BRUNOVSKY,428.622810


In [135]:
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=True).reset_index(drop=True)[0:20]

,autor,priemerny_rocny_narast_v_cene
0,BARTFAY TIBOR,-403.880303
1,MILOS SIMURDA,-178.769780
2,JULIUS KOLLER,-141.069214
3,IMRO WEINER-KRAL,-104.859096
4,JAN ZELIBSKY,-101.202749
5,HAVRILLA VLADIMIR,-98.458698
6,PAVOL POTOCEK,-88.391473
7,ILECKO JOZEF,-72.323358
8,JOZEF KOLLAR,-54.459738
9,BARTFAY JULIUS,-46.766457


In [136]:
# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].mean().reset_index()

# Filter authors with at least 50 rows
selected_authors = ['MILAN PASTEKA', 'FILKO STANO', 'LALUHA MILAN']
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Create Bokeh ColumnDataSource
source = ColumnDataSource(df_avg_filtered)

# Plotting
output_notebook()
p = figure(plot_width=800, plot_height=400, title='Average Price Trends Over Time for Each Name',
           x_axis_label='Year', y_axis_label='Average Price')

# Add a line for each autor
for color, autor in zip(Category10[6], df_avg_filtered['autor'].unique()):
    autor_data = df_avg_filtered[df_avg_filtered['autor'] == autor]
    autor_source = ColumnDataSource(autor_data)
    p.line(x='rok_drazby', y='konecna_cena', source=autor_source, line_width=2, color=color, legend_label=autor)
    p.circle(x='rok_drazby', y='konecna_cena', source=autor_source, size=8, color=color, legend_label=autor)

p.legend.location = 'top_left'
show(p)

Loading BokehJS ...